In [332]:
%%capture
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import networkx as nx
import pandas as pd
import sys
sys.path.append('/home/wrwt/Programming/pygraphmodels')
import graphmodels as gm
from itertools import permutations
from graphmodels import MatrixGraph, DGM
%matplotlib inline
%load_ext line_profiler

In [333]:
x = np.random.randint(0, 2, size=10000)
y = np.random.randint(0, 2, size=10000)

In [334]:
df = pd.DataFrame(data={'x': x, 'y': y})

In [335]:
gm.information.discrete_entropy(df[['x']])

0.69314268055321326

In [336]:
gm.information.discrete_entropy(df[['y']])

0.69312268035995606

In [337]:
gm.information.discrete_entropy(df[['x', 'y']])

1.3862068093365769

In [338]:
2 * -0.5 * np.log(0.5)

0.69314718055994529

In [339]:
4 * -0.5 * np.log(0.5)

1.3862943611198906

In [340]:
from os import listdir
import os.path
NETWORKS_PATH = '/home/wrwt/Programming/pygraphmodels/networks/'
network_filenames = listdir(NETWORKS_PATH)
true_dgm = gm.DGM.read(os.path.join(NETWORKS_PATH, 'earthquake.bif'))
true_dgm.draw()

In [341]:
data = true_dgm.rvs(size=100000)

In [342]:
gm.information.discrete_entropy(data[['Earthquake', 'Burglary', 'Alarm', 'MaryCalls']])

0.23755339286258073

In [343]:
gm.information.discrete_entropy(data[['Earthquake', 'Burglary']])

0.15188146801726018

In [344]:
gm.information.discrete_entropy(data[['Earthquake']]) + gm.information.discrete_entropy(data[['Burglary']])

0.15191010235216457

In [528]:
from itertools import repeat, chain
class SubsetGraph(nx.DiGraph):
    def __init__(self):
        nx.DiGraph.__init__(self)
        self.lower_cache = {}
        self.upper_cache = {}
        
    def add_subset(self, subs, f=None):
        subs = frozenset(subs)
        if subs in self.nodes():
            return 
        self.add_node(subs, f=f)
        for node in list(self.nodes()):
            if node > subs and all(not child > subs for child in self.successors(node)):
                self.add_edge(node, subs)
        
        for pa, ch in list(self.edges()):
            if subs < pa and ch < subs:
                self.remove_edge(pa, ch)
                self.add_edge(subs, ch)
        for ch in list(self.nodes()):
            if ch < subs and all(not parent <= subs for parent in self.predecessors(ch)):
                self.add_edge(subs, ch)
                
    def get_parents(self, subs):
        subs = frozenset(subs)
        if subs in self.nodes():
            return self.predecessors(subs)
        results = []
        for node in list(self.nodes()):
            if node > subs and all(not child > subs for child in self.successors(node)):
                results.append(node)
        return results
    
    def get_children(self, subs):
        subs = frozenset(subs)
        if subs in self.nodes():
            return self.successors(subs)
        results = set()
        for pa, ch in list(self.edges()):
            if subs < pa and ch < subs:
                results.add(ch)
        for ch in list(self.nodes()):
            if ch < subs and all(not parent <= subs for parent in self.predecessors(ch)):
                results.add(ch)
        return results
    
    def upper_bound(self, subs):
        subs = frozenset(subs)
        
        if subs in self.upper_cache:
            current = self.upper_cache[subs]
        else:
            current = np.inf
        
        if subs in self.node:
            self.upper_cache[subs] = self.node[subs]['f']
            yield self.node[subs]['f']
            return
        
        yield current
        for ch in self.get_children(subs):
            for ub in self.upper_bound(subs - ch):
                temp = self.node[ch]['f'] + ub
                if temp < current:
                    current = temp
                    self.upper_cache[subs] = current
                    yield current
                    
        for pa in self.get_parents(subs):
            temp = self.node[pa]['f']
            if temp < current:
                current = temp
                self.upper_cache[subs] = current
                yield current
        
    def lower_bound1(self, subs):        
        current = 0
        
        for ch in self.get_children(subs):
            for lb in self.lower_bound(subs - ch):
                temp = max(self.node[ch]['f'], lb)
                if temp > current:
                    current = temp
                    self.lower_cache[subs] = current
                    yield current
    
    def lower_bound2(self, subs):
        current = 0
        
        for pa in self.get_parents(subs):
            for ub in self.upper_bound(pa - subs):
                temp = self.node[pa]['f'] - ub
                if temp > current:
                    current = temp
                    self.lower_cache[subs] = current
                    yield current
    
    def lower_bound(self, subs):
        subs = frozenset(subs)
        
        if subs in self.lower_cache:
            current = self.lower_cache[subs]
        else:
            current = 0
        yield current
        
        if subs in self.node:
            self.lower_cache[subs] = self.node[subs]['f']
            yield self.node[subs]['f']
            return
        
        for lb in self.lower_bound1(subs):
            if lb > current:
                current = lb
                yield current
        for lb in self.lower_bound2(subs):
            if lb > current:
                current = lb
                yield current
        

In [445]:
sg = SubsetGraph()
sg.add_subset([1], 0.5)
sg.add_subset([3, 4], 1.1)
sg.add_subset([1, 2], 0.7)
sg.add_subset([2], 0.5)
sg.add_subset([3], 0.3)
sg.add_subset([4], 1.0)
sg.add_subset([1, 2, 3], 0.9)
sg.add_subset([1, 2, 3, 4], 1.9)
#sg.lower_bound([1, 3, 4]), sg.upper_bound([1, 3, 4])

In [448]:
for lb in sg.lower_bound([1, 3, 4]):
    print(lb)

1.4


In [449]:
for ub in sg.upper_bound([1, 3, 4]):
    print(ub)

1.6


In [410]:
sg = SubsetGraph()
sg.add_subset([1], 0.3)
sg.add_subset([2], 0.4)
sg.add_subset([3], 0.5)
sg.add_subset([4], 0.6)
sg.add_subset([1, 2], 0.55)
sg.add_subset([2, 3], 0.7)
sg.add_subset([3, 4], 1.0)
list(sg.lower_bound([1, 4])), list(sg.upper_bound([1, 4]))

([0, 0.6], [inf, 0.8999999999999999])

In [411]:
#data = true_dgm.rvs(10000)

In [412]:
sg = SubsetGraph()
ee = gm.EntropyEstimator(gm.MatrixGraph.from_networkx_DiGraph(true_dgm, order=data.columns), data)

In [413]:
def add_subs(*subs):
    subs = frozenset(subs)
    f = ee([name in subs for name in data.columns])
    sg.add_subset(subs, f)

In [414]:
def entr(*subs):
    return ee([name in subs for name in data.columns])

In [415]:
def bounds(*subs):
    return sg.lower_bound(frozenset(subs)), sg.upper_bound(frozenset(subs))

In [416]:
for var in data.columns:
    add_subs(var)

In [417]:
add_subs('Burglary', 'Earthquake', 'MaryCalls', 'JohnCalls')

In [418]:
%time entr('Burglary', 'MaryCalls')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 83.9 µs


0

In [419]:
#bounds('Alarm', 'Burglary')

In [420]:
#bounds('Alarm', 'Earthquake')

In [421]:
#bounds('Alarm', 'Earthquake', 'MaryCalls')[1] - bounds('Alarm', 'Earthquake')[0]

In [422]:
#bounds('Alarm', 'Earthquake', 'MaryCalls')[0] - bounds('Alarm', 'Earthquake')[1]

In [423]:
import numpy as np
from graphmodels.information import discrete_entropy


class EntropyEstimator:
    def __init__(self, graph, data):
        self.graph = graph
        self.data = data
        self.cache = {}

    def _footprint(self, nodes):
        return tuple(nodes)

    def __call__(self, nodes):
        fp = self._footprint(nodes)
        if fp in self.cache:
            return self.cache[fp]

        if np.any(nodes):
            subdata = self.data[self.data.columns[nodes]]
            result = discrete_entropy(subdata)
        else:
            result = 0

        self.cache[fp] = result
        return result


class InformationEstimator:
    def __init__(self, graph, data):
        self.graph = graph
        self.entropy_estimator = EntropyEstimator(graph, data)
        self.chosen = []

    def __call__(self, node, parents):
        parents = np.array(parents, dtype=bool)
        h1 = self.entropy_estimator(parents)
        temp = np.zeros(len(parents), dtype=bool)
        temp[node] = True
        h2 = self.entropy_estimator(temp)
        parents[node] = True
        h12 = self.entropy_estimator(parents)
        return h1 + h2 - h12


class ScoreBIC:
    def __init__(self, graph, data):
        self.graph = graph
        self.data = data
        self.n_values = np.asarray([len(self.data[column].value_counts()) for column in self.data.columns])
        self.mi_estimator = InformationEstimator(graph, data)

    def __call__(self, node, parents):
        parents = np.asarray(parents, dtype=bool)
        k = self.n_values[node]*np.prod(self.n_values[parents]) - 1
        n = self.data.shape[0]
        l = n*self.mi_estimator(node, parents)

        result = l - 0.5 * np.log(n) * k
        return result

    def total(self):
        score = 0.
        for node in self.graph.nodes():
            pa = self.graph.adj[:, node].copy()
            score += self(node, pa)
        return score

In [424]:
def interzip(*iterators):
    iterators = [it.__iter__() for it in iterators]
    finished = [False] * len(iterators)
    last = [None] * len(iterators)
    while True:
        for i, iterator in enumerate(iterators):
            try:
                last[i] = iterator.next()
            except StopIteration:
                finished[i] = True
        if all(finished):
            return
        yield tuple(last)

In [425]:
list(interzip([1, 2, 3], ['a', 'b'], {'str'}))

[(1, 'a', 'str'), (2, 'b', 'str'), (3, 'b', 'str')]

In [426]:
def last(it):
    for res in it:
        pass
    return res

last(interzip([1, 2, 3], ['a', 'b'], {'str'}))

(3, 'b', 'str')

In [427]:
class AdvancedEntropyEstimator:
    def __init__(self, graph, data):
        self.graph = graph
        self.data = data
        self.sg = SubsetGraph()
        self._initialize_sg()

    def _initialize_sg(self):
        vec = np.asarray([False] * len(self.data.columns), dtype=bool)
        for i, column in enumerate(self.data.columns):
            vec[i] = True
            self.__call__(vec)
            vec[i] = False
        
    def _key(self, subs):
        return frozenset([name for name, included in zip(data.columns, subs) if included])
    
    def _add_subs(self, subs, f):
        self.sg.add_subset(subs, f)
    
    def __call__(self, nodes):
        if not np.any(nodes):
            return 0
        key = self._key(nodes)
        if key in self.sg.nodes():
            return self.sg.node[key]['f']
        subdata = self.data[self.data.columns[nodes]]
        result = discrete_entropy(subdata)
        self._add_subs(key, f=result)
        return result
    
    def lower_bound(self, nodes):
        if not np.any(nodes):
            return [0]
        key = self._key(nodes)
        return self.sg.lower_bound(key)
    
    def upper_bound(self, nodes):
        if not np.any(nodes):
            return [0]
        key = self._key(nodes)
        return self.sg.upper_bound(key)
    
class AdvancedInformationEstimator:
    def __init__(self, graph, data):
        self.graph = graph
        self.entropy_estimator = AdvancedEntropyEstimator(graph, data)
        self.chosen = []

    def __call__(self, node, parents):
        parents = np.array(parents, dtype=bool)
        h1 = self.entropy_estimator(parents)
        temp = np.zeros(len(parents), dtype=bool)
        temp[node] = True
        h2 = self.entropy_estimator(temp)
        parents[node] = True
        h12 = self.entropy_estimator(parents)
        return h1 + h2 - h12
    
    def lower_bound(self, node, parents):
        parents = np.array(parents, dtype=bool)
        h1 = self.entropy_estimator.lower_bound(parents)
        temp = np.zeros(len(parents), dtype=bool)
        temp[node] = True
        h2 = self.entropy_estimator.lower_bound(temp)
        parents[node] = True
        h12 = self.entropy_estimator.upper_bound(parents)
        for current_h1, current_h2, current_h12 in interzip(h1, h2, h12):
            yield current_h1 + current_h2 - current_h12
        
    def upper_bound(self, node, parents):
        parents = np.array(parents, dtype=bool)
        h1 = self.entropy_estimator.upper_bound(parents)
        temp = np.zeros(len(parents), dtype=bool)
        temp[node] = True
        h2 = self.entropy_estimator.upper_bound(temp)
        parents[node] = True
        h12 = self.entropy_estimator.lower_bound(parents)
        for current_h1, current_h2, current_h12 in interzip(h1, h2, h12):
            yield current_h1 + current_h2 - current_h12
    
class AdvancedScoreBIC:
    def __init__(self, graph, data):
        self.graph = graph
        self.data = data
        self.n_values = np.asarray([len(self.data[column].value_counts()) for column in self.data.columns])
        self.mi_estimator = AdvancedInformationEstimator(graph, data)

    def __call__(self, node, parents, option=None):
        parents = np.asarray(parents, dtype=bool)
        k = self.n_values[node]*np.prod(self.n_values[parents]) - 1
        n = self.data.shape[0]
        
        l = n*self.mi_estimator(node, parents)

        result = l - 0.5 * np.log(n) * k
        return result
    
    def lower_bound(self, node, parents):
        parents = np.asarray(parents, dtype=bool)
        k = self.n_values[node]*np.prod(self.n_values[parents]) - 1
        n = self.data.shape[0]
        l_iter = self.mi_estimator.lower_bound(node, parents)

        for l in l_iter:
            yield n*l - 0.5 * np.log(n) * k

    def upper_bound(self, node, parents):
        parents = np.asarray(parents, dtype=bool)
        k = self.n_values[node]*np.prod(self.n_values[parents]) - 1
        n = self.data.shape[0]
        l_iter = self.mi_estimator.upper_bound(node, parents)

        for l in l_iter:
            yield n*l - 0.5 * np.log(n) * k
    
    def total(self):
        score = 0.
        for node in self.graph.nodes():
            pa = self.graph.adj[:, node].copy()
            score += self(node, pa)
        return score

In [428]:
from graphmodels import LocalOperation

class AdvancedAddEdge(LocalOperation):
    def __init__(self, graph, fscore, src, dst):
        LocalOperation.__init__(self, graph, fscore)
        self.src = src
        self.dst = dst

    def do(self):
        if self.graph.adj[self.src, self.dst]:
            raise InvalidOperation()
        self.graph.adj[self.src, self.dst] = 1
        if not self.graph.is_acyclic():
            self.graph.adj[self.src, self.dst] = 0
            raise InvalidOperation()
        return self

    def undo(self):
        self.graph.adj[self.src, self.dst] = 0
        return self

    def score(self):
        pa = self.graph.adj[:, self.dst].copy()
        score = -self.fscore(self.dst, pa)
        pa[self.src] = 1
        score += self.fscore(self.dst, pa)
        return score
    
    def lower_bound(self):
        pa = self.graph.adj[:, self.dst].copy()
        ub_iter = self.fscore.upper_bound(self.dst, pa)
        pa[self.src] = 1
        lb_iter = self.fscore.lower_bound(self.dst, pa)
        
        for lb, ub in interzip(lb_iter, ub_iter):
            yield lb - ub
    
    def upper_bound(self):
        pa = self.graph.adj[:, self.dst].copy()
        lb_iter = self.fscore.lower_bound(self.dst, pa)
        pa[self.src] = 1
        ub_iter = self.fscore.upper_bound(self.dst, pa)
        
        for lb, ub in interzip(lb_iter, ub_iter):
            yield ub - lb
        

class AdvancedRemoveEdge(LocalOperation):
    def __init__(self, graph, fscore, src, dst):
        LocalOperation.__init__(self, graph, fscore)
        self.src = src
        self.dst = dst

    def do(self):
        if not self.graph.adj[self.src, self.dst]:
            raise InvalidOperation()
        self.graph.adj[self.src, self.dst] = 0
        return self

    def undo(self):
        self.graph.adj[self.src, self.dst] = 1
        return self

    def score(self):
        pa = self.graph.adj[:, self.dst].copy()
        score = -self.fscore(self.dst, pa)
        pa[self.src] = 0
        score += self.fscore(self.dst, pa)
        return score
    
    def lower_bound(self):
        pa = self.graph.adj[:, self.dst].copy()
        ub_iter = self.fscore.upper_bound(self.dst, pa)
        pa[self.src] = 0
        lb_iter = self.fscore.lower_bound(self.dst, pa)
        
        for lb, ub in interzip(lb_iter, ub_iter):
            yield lb - ub

    def upper_bound(self):
        pa = self.graph.adj[:, self.dst].copy()
        lb_iter = self.fscore.lower_bound(self.dst, pa)
        pa[self.src] = 0
        ub_iter = self.fscore.upper_bound(self.dst, pa)
        
        for lb, ub in interzip(lb_iter, ub_iter):
            yield ub - lb

class AdvancedReverseEdge(LocalOperation):
    def __init__(self, graph, fscore, src, dst):
        LocalOperation.__init__(self, graph, fscore)
        self.src = src
        self.dst = dst

    def do(self):
        if not self.graph.adj[self.src, self.dst]:
            raise InvalidOperation()
        self.graph.adj[self.src, self.dst] = 0
        self.graph.adj[self.dst, self.src] = 1
        if not self.graph.is_acyclic():
            self.graph.adj[self.src, self.dst] = 1
            self.graph.adj[self.dst, self.src] = 0
            raise InvalidOperation()
        return self

    def undo(self):
        self.graph.adj[self.src, self.dst] = 1
        self.graph.adj[self.dst, self.src] = 0
        return self

    def score(self):
        pa = self.graph.adj[:, self.dst].copy()
        score = -self.fscore(self.dst, pa)
        pa[self.src] = 0
        score += self.fscore(self.dst, pa)

        pa = self.graph.adj[:, self.src].copy()
        score -= self.fscore(self.src, pa)
        pa[self.dst] = 1
        score += self.fscore(self.src, pa)
        return score
    
    def lower_bound(self):
        pa = self.graph.adj[:, self.dst].copy()
        ub_iter1 = self.fscore.upper_bound(self.dst, pa)
        pa[self.src] = 0
        lb_iter1 = self.fscore.lower_bound(self.dst, pa)

        pa = self.graph.adj[:, self.src].copy()
        ub_iter2 = self.fscore.upper_bound(self.src, pa)
        pa[self.dst] = 1
        lb_iter2 = self.fscore.lower_bound(self.src, pa)
        
        for lb1, lb2, ub1, ub2 in interzip(lb_iter1, lb_iter2, ub_iter1, ub_iter2):
            yield lb1 + lb2 - ub1 - ub2
    
    def upper_bound(self):
        pa = self.graph.adj[:, self.dst].copy()
        lb_iter1 = self.fscore.lower_bound(self.dst, pa)
        pa[self.src] = 0
        ub_iter1 = self.fscore.upper_bound(self.dst, pa)

        pa = self.graph.adj[:, self.src].copy()
        lb_iter2 = self.fscore.lower_bound(self.src, pa)
        pa[self.dst] = 1
        ub_iter2 = self.fscore.upper_bound(self.src, pa)
        
        for lb1, lb2, ub1, ub2 in interzip(lb_iter1, lb_iter2, ub_iter1, ub_iter2):
            yield ub1 + ub2 - lb1 - lb2

In [514]:
from graphmodels import AddEdge, RemoveEdge, ReverseEdge, InvalidOperation
class AdvancedGreedySearch:
    def __init__(self, data, cls_score):
        graph = nx.DiGraph()
        graph.add_nodes_from(data.columns)
        graph = MatrixGraph.from_networkx_DiGraph(graph, order=data.columns)
        self.graph = graph
        self.fscore = cls_score(graph, data)
        self.tscore = ScoreBIC(graph, data)

        self.ops = []
        self.ops += [AdvancedAddEdge(graph, self.fscore, u, v) for u, v in permutations(graph.nodes(), 2)]
        self.ops += [AdvancedRemoveEdge(graph, self.fscore, u, v) for u, v in permutations(graph.nodes(), 2)]
        self.ops += [AdvancedReverseEdge(graph, self.fscore, u, v) for u, v in permutations(graph.nodes(), 2)]
    
    
    def iteration(self):
        #ops.sort(reverse=True, key=lambda x: x[1] + x[2])
        
        best_op = None
        best_score = 0.
        
        skipped = 0
        instaskipped = 0
        
        for i, op in enumerate(self.ops):
            skip = False
            for j, ub in enumerate(op.upper_bound()):
                #print(ub)
                if ub < best_score:
                    if j == 0:
                        instaskipped += 1
                    skipped += 1
                    break
            if skip:
                continue
                
            try:
                current_score = op.score()
                op.do()
                if current_score > best_score:
                    best_score = current_score
                    best_op = op
                op.undo()
            except InvalidOperation:
                pass
        
        print 'done {}, skippped {}, instaskipped {}, operations out of {}'.format(len(self.ops) - skipped, 
                                                                 skipped, instaskipped, len(self.ops))
        if best_op is None or best_op.score() < 1e-5:
            return True
        best_op.do()
        return False

    def __call__(self, max_iter=40, verbose=True):
        counter = 0
        while not self.iteration() and counter < max_iter:
            if verbose:
                print(self.fscore.total())
            counter += 1
        return DGM(self.graph.to_networkx_DiGraph())

In [466]:
from os import listdir
import os.path
NETWORKS_PATH = '/home/wrwt/Programming/pygraphmodels/networks/'
network_filenames = listdir(NETWORKS_PATH)
true_dgm = gm.DGM.read(os.path.join(NETWORKS_PATH, 'alarm.bif'))
true_dgm.draw()

In [454]:
data = true_dgm.rvs(size=100000)

In [537]:
gs = AdvancedGreedySearch(data, AdvancedScoreBIC)

In [539]:
%lprun -f interzip res = gs(max_iter=10)

done 13, skippped 3983, instaskipped 3920, operations out of 3996
542991.27202
done 31, skippped 3965, instaskipped 3923, operations out of 3996
574523.353328
done 31, skippped 3965, instaskipped 3915, operations out of 3996
605817.885984
done 4, skippped 3992, instaskipped 3948, operations out of 3996
637068.739386
done 35, skippped 3961, instaskipped 3923, operations out of 3996
665799.515822
done 6, skippped 3990, instaskipped 3926, operations out of 3996
692240.096555
done 29, skippped 3967, instaskipped 3937, operations out of 3996
717533.432651
done 33, skippped 3963, instaskipped 3931, operations out of 3996
742537.742507
done 35, skippped 3961, instaskipped 3700, operations out of 3996
767492.144259
done 35, skippped 3961, instaskipped 3907, operations out of 3996
785270.60568
done 35, skippped 3961, instaskipped 3927, operations out of 3996


In [517]:
from graphmodels import GreedySearch
gs = GreedySearch(data, ScoreBIC)

In [518]:
%%time
res = gs(max_iter=100)

61153.4389484
114283.117973
166866.83307
218163.117983
265279.030207
310787.989394
356074.948469
399560.564894
439657.154877
475858.513705
511264.439351
542991.27202
574523.353328
605817.885984
637068.739386
665799.515822
692240.096555
717533.432651
742537.742507
767492.144259
785270.60568
805124.520707
822022.425941
836942.009742
851788.035307
866189.848769
880395.126555
893666.308202
905224.118387
916553.274549
927101.531052
936678.903457
944964.19197
952141.759744
958384.564726
964048.66509
968615.318324
972702.17352
976706.063158
980552.107773
983926.585941
986941.003198
989892.250102
992477.480032
994708.028574
996902.016773
998891.650055
1000415.3148
1001550.17535
1002643.0521
1002859.04319
1003049.33724
1003176.72049
1003268.82389
1003379.72238
1003429.58032
CPU times: user 2min 29s, sys: 92 ms, total: 2min 29s
Wall time: 2min 29s


In [ ]:
gs.fscore.mi_estimator.entropy_estimator.sg.nodes(data=True)

In [ ]:
data.columns

In [ ]:
vec = [False] * len(data.columns)
#vec[list(data.columns).index('CVP')] = True
vec[list(data.columns).index('LVEDVOLUME')] = True
vec[8] = True
vec[9] = True
vec[11] = True
vec[20] = True

In [ ]:
%%time
gs.fscore.lower_bound(list(data.columns).index('CVP'), vec)

In [ ]:
gs.fscore.upper_bound(list(data.columns).index('CVP'), vec)

In [ ]:
%%time
gs.fscore(list(data.columns).index('CVP'), vec)

In [ ]:
len(gs.fscore.mi_estimator.entropy_estimator.sg.nodes())